In [44]:
grado_ht = {'joule':0.35,'BC':0.28,'caldera_GN':0.15,'butano':0.15,'nada':0.07}
grado_cl = {'split':0.1,'BC':0.43,'multi_split':0.04,'nada':0.43}
grado_dhw ={'caldera_GN':0.64,'butano':0.1,'BC':0, 'joule':0.26}

coef_NRPE={'ele': 2.937, 'GN': 1.190, 'butano': 1.201}
EER = 2
COP=2 
COP_DHW=1.7
eta_cal=0.7


In [56]:
caldera = 0.006097*(grado_dhw['caldera_GN']/eta_cal*coef_NRPE['GN'])
caldera

0.006633536000000001

In [57]:
butano = 0.006097*grado_dhw['butano']/eta_cal*(coef_NRPE['butano'])

butano


0.0010460710000000002

In [58]:
bc = 0.006097* (grado_dhw['BC']/COP_DHW*coef_NRPE['ele'])
bc

0.0

In [59]:
joule = 0.006097 * (grado_dhw['joule']/1 * coef_NRPE['ele'])
joule

0.00465579114

In [52]:
0.14*((grado_dhw['caldera_GN']/eta_cal*coef_NRPE['GN'])+(grado_dhw['butano']/eta_cal*coef_NRPE['butano'])+(grado_dhw['BC']/COP_DHW*coef_NRPE['ele'])+(grado_dhw['joule']/1 * coef_NRPE['ele']))

0.2832468000000001

In [60]:
joule+bc+butano+caldera

0.01233539814

In [55]:
0.64/0.7*1.19 + 0.1/0.7*1.201 + 0 + 0.26/1*2.968

2.0312514285714287

In [ ]:
0.64/0.7*1.19 + 0.14/0.7*1.201 +0 + 0 = 1.7022

In [1]:
import pandas as pd

from interactors import analysisActive,  analysisTRNSYS2, executionCost2, executionCost
from repositories import csvTRNSYS2
from utils import meanCalculation2, energyScenarios2
from SQL_statements import database_connection, getSurfacesTemp
from sqlalchemy import text
from repositories import csvTRNSYS, CMH
from utils import mean_calculation2

In [8]:
engine = database_connection.engine
Arquetipo = 'FE35'
with open('SQL_statements/materials.sql', 'r') as file:
    materials_query = file.read()

        # Execute the query with the parcela variable
with engine.connect() as connection:
    query = text(materials_query)
    materials = pd.read_sql_query(query, connection, params={"Arquetipo":Arquetipo })
    
materials[['convecional_eco','opcion']] = materials[['convecional_eco','opcion']].astype('int')
materials

,tipus,convecional_eco,opcion,espesor,espesor_mm,cost
0,coberta,2,2,R10,100.0,151.8115
1,coberta,1,1,R6,60.0,142.3500
2,mur,1,1,W8,80.0,92.7360
3,coberta,1,1,R10,100.0,148.8100
4,mur,2,1,W6,60.0,137.5400
5,coberta,1,1,R14,140.0,155.1300
6,coberta,2,1,R10,100.0,169.9930
7,mur,2,2,W6,60.0,137.5400
8,finestra,1,0,0,0.0,676.9700
9,coberta,1,1,R12,120.0,151.9500


In [9]:
materials.loc[(materials['tipus'] == 'mur') & (materials['espesor'] == 'R5')]

,tipus,convecional_eco,opcion,espesor,espesor_mm,cost


In [11]:
materials.loc[(materials['tipus'] == 'coberta')]

,tipus,convecional_eco,opcion,espesor,espesor_mm,cost
0,coberta,2,2,R10,100.0,151.8115
1,coberta,1,1,R6,60.0,142.3500
3,coberta,1,1,R10,100.0,148.8100
5,coberta,1,1,R14,140.0,155.1300
6,coberta,2,1,R10,100.0,169.9930
9,coberta,1,1,R12,120.0,151.9500
11,coberta,1,2,R10,100.0,171.4075
12,coberta,1,1,R8,80.0,145.6800


In [12]:
import pickle

# Specify the path to your PKL file
file_path = "C:/arxius/PV_estimation.pkl"

# Open the file in read-binary mode
with open(file_path, 'rb') as file:
    # Load the data from the file
    PV_estimation = pickle.load(file)

In [18]:
filtered_keys = [key for key in PV_estimation.keys() if key.endswith('R5')]
filtered_keys

['P221_0_W6_R5',
 'P221_0_W8_R5',
 'P221_180_W6_R5',
 'P221_180_W8_R5',
 'P221_270_W6_R5',
 'P221_270_W8_R5',
 'P221_90_W6_R5',
 'P221_90_W8_R5',
 'P222_0_W6_R5',
 'P222_0_W8_R5',
 'P222_180_W6_R5',
 'P222_180_W8_R5',
 'P222_270_W6_R5',
 'P222_270_W8_R5',
 'P222_90_W6_R5',
 'P222_90_W8_R5']

In [19]:
updated_PV_estimation = {key.replace('R5', 'R10'): value for key, value in PV_estimation.items()}
updated_PV_estimation

{'BC0_0': {'Split_rf+PV_yes+BC_yes': 13.436042178563591,
  'Split_rf+PV_yes+BC_no': 11.446425736084237,
  'Split_rf+PV_no+BC_yes': 0,
  'Split_rf+PV_no+BC_no': 0,
  'Split_nrf+PV_yes+BC_yes': 24.022309959577175,
  'Split_nrf+PV_yes+BC_no': 22.315351874901243,
  'Split_nrf+PV_no+BC_yes': 0,
  'Split_nrf+PV_no+BC_no': 0},
 'BC0_180': {'Split_rf+PV_yes+BC_yes': 13.436042178563591,
  'Split_rf+PV_yes+BC_no': 11.446425736084237,
  'Split_rf+PV_no+BC_yes': 0,
  'Split_rf+PV_no+BC_no': 0,
  'Split_nrf+PV_yes+BC_yes': 24.022309959577175,
  'Split_nrf+PV_yes+BC_no': 22.315351874901243,
  'Split_nrf+PV_no+BC_yes': 0,
  'Split_nrf+PV_no+BC_no': 0},
 'BC0_270': {'Split_rf+PV_yes+BC_yes': 13.436042178563591,
  'Split_rf+PV_yes+BC_no': 11.446425736084237,
  'Split_rf+PV_no+BC_yes': 0,
  'Split_rf+PV_no+BC_no': 0,
  'Split_nrf+PV_yes+BC_yes': 24.022309959577175,
  'Split_nrf+PV_yes+BC_no': 22.315351874901243,
  'Split_nrf+PV_no+BC_yes': 0,
  'Split_nrf+PV_no+BC_no': 0},
 'BC0_90': {'Split_rf+PV_yes+B

In [20]:
with open('C:/arxius/PV_estimation.pkl','wb') as file:
    pickle.dump(updated_PV_estimation, file)

In [34]:
file_path = "C:/arxius/buildingCost_active.pkl"

# Open the file in read-binary mode
with open(file_path, 'rb') as file:
    # Load the data from the file
    buildingCost_active = pickle.load(file)
    
buildingCost_active

{'BC0_0': {},
 'BC0_180': {},
 'BC0_270': {},
 'BC0_90': {},
 'P111_0_W8_R10': {'Split_rf+PV_yes+BC_yes': {'1701521DD7810B': {'wall_PEC': 82.53504,
    'roof_PEC': 132.4409,
    'cost_wall': 92.73599999999999,
    'cost_roof': 148.81,
    'wall_type': 'W8',
    'roof_type': 'R10',
    'pattern': 'P111'}},
  'Split_rf+PV_yes+BC_no': {'1701521DD7810B': {'wall_PEC': 82.53504,
    'roof_PEC': 132.4409,
    'cost_wall': 92.73599999999999,
    'cost_roof': 148.81,
    'wall_type': 'W8',
    'roof_type': 'R10',
    'pattern': 'P111'}},
  'Split_rf+PV_no+BC_yes': {'1701521DD7810B': {'wall_PEC': 82.53504,
    'roof_PEC': 132.4409,
    'cost_wall': 92.73599999999999,
    'cost_roof': 148.81,
    'wall_type': 'W8',
    'roof_type': 'R10',
    'pattern': 'P111'}},
  'Split_rf+PV_no+BC_no': {'1701521DD7810B': {'wall_PEC': 82.53504,
    'roof_PEC': 132.4409,
    'cost_wall': 92.73599999999999,
    'cost_roof': 148.81,
    'wall_type': 'W8',
    'roof_type': 'R10',
    'pattern': 'P111'}},
  'Split_n

In [35]:
buildingCost_active = pd.DataFrame(buildingCost_active)

In [36]:
len(buildingCost_active.columns[buildingCost_active.isnull().any()])

4

In [37]:
len(buildingCost_active.columns)

84

In [38]:
buildingCost_active

,BC0_0,BC0_180,BC0_270,BC0_90,P111_0_W8_R10,P111_0_W8_R12,P111_0_W8_R14,P111_0_W8_R6,P111_0_W8_R8,P111_180_W8_R10,...,P221_90_W6_R10,P221_90_W8_R10,P222_0_W6_R10,P222_0_W8_R10,P222_180_W6_R10,P222_180_W8_R10,P222_270_W6_R10,P222_270_W8_R10,P222_90_W6_R10,P222_90_W8_R10
Split_rf+PV_yes+BC_yes,NaN,NaN,NaN,NaN,"{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...",...,{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro...",{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro...",{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro...",{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro...",{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro..."
Split_rf+PV_yes+BC_no,NaN,NaN,NaN,NaN,"{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...",...,{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro...",{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro...",{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro...",{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro...",{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro..."
Split_rf+PV_no+BC_yes,NaN,NaN,NaN,NaN,"{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...",...,{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro...",{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro...",{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro...",{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro...",{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro..."
Split_rf+PV_no+BC_no,NaN,NaN,NaN,NaN,"{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...",...,{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro...",{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro...",{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro...",{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro...",{'1701521DD7810B': {'wall_PEC': 122.4105999999...,"{'1701521DD7810B': {'wall_PEC': 137.61981, 'ro..."
Split_nrf+PV_yes+BC_yes,NaN,NaN,NaN,NaN,"{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall_PEC': 82.53504, 'roo...","{'1701521DD7810B': {'wall